In [1]:
import os
import shutil
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


In [2]:
class DataLoader:
    def __init__(self,path):
        self.path = path
        
    def get_list_files(self):
        
        return sorted(os.listdir(self.path))
    
    def get_random_images(self,channel,which_chan):
        random_brain_dir = random.choice(self.get_list_files())
        exact_path = self.path + '/' + random_brain_dir

        number_of_image = random_brain_dir[-5:]
        
        image_flair = nib.load(f'{exact_path}/BraTS2021_{number_of_image}_flair.nii.gz').get_fdata()
        image_seg = nib.load(f'{exact_path}/BraTS2021_{number_of_image}_seg.nii.gz').get_fdata()
        image_t1 = nib.load(f'{exact_path}/BraTS2021_{number_of_image}_t1.nii.gz').get_fdata()
        image_t1ce = nib.load(f'{exact_path}/BraTS2021_{number_of_image}_t1ce.nii.gz').get_fdata()
        image_t2 = nib.load(f'{exact_path}/BraTS2021_{number_of_image}_t2.nii.gz').get_fdata()
        
        which_channel =which_chan
        if which_channel == 1:
            mid_slice_flair = image_flair[channel,:,:]
            mid_slice_t1 = image_t1[channel,:,:]
            mid_slice_t1ce = image_t1ce[channel,:,:]
            mid_slice_t2 = image_t2[channel,:,:]
        
        elif which_channel == 2:
            
            mid_slice_flair = image_flair[:,channel,:]
            mid_slice_t1 = image_t1[:,channel,:]
            mid_slice_t1ce = image_t1ce[:,channel,:]
            mid_slice_t2 = image_t2[:,channel,:]
            
            
            
        elif which_channel == 3:
            mid_slice_flair = image_flair[:,:,channel]
            mid_slice_t1 = image_t1[:,:,channel]
            mid_slice_t1ce = image_t1ce[:,:,channel]
            mid_slice_t2 = image_t2[:,:,channel]
            
            
        
        fig,axes = plt.subplots(2, 3, figsize=(30, 25))    

        fig.suptitle(f'Mozg nr {number_of_image}', fontsize=40)
        mid_slice_seg = image_seg[:,:,channel]
        fig.patch.set_facecolor('white')

        axes[0,0].imshow(mid_slice_flair.T,cmap='gray', origin='lower')
        axes[0,0].set_title('Flair')
            
        axes[0,1].imshow(mid_slice_t1.T,cmap='gray', origin='lower')
        axes[0,1].set_title('t1')
            
        axes[0,2].imshow(mid_slice_t1ce.T,cmap='gray', origin='lower')
        axes[0,2].set_title('t1ce')
            
        axes[1,0].imshow(mid_slice_t2.T,cmap='gray', origin='lower')
        axes[1,0].set_title('t2')
            
        axes[1,1].imshow(mid_slice_seg.T,cmap='gray', origin='lower')
        axes[1,1].set_title('t1seg')
            
        axes[1,2].imshow(mid_slice_seg.T, origin='lower')
        axes[1,2].set_title('t1seg')
            
        
    def get_data_train(self):
        path_dirs = self.path
        files = self.get_list_files()[:1200]
        
        brain = []
        seg = []
  
        for file in files:
            image_seg = nib.load(f'{self.path}/{file}/BraTS2021_{file[-5:]}_seg.nii.gz').get_fdata()            
            
            mid_slice_seg = image_seg[:,:,90]
            
            seg.append(np.array(mid_slice_seg))
           
            
            
        for file in files:
            image_t1ce = nib.load(f'{self.path}/{file}/BraTS2021_{file[-5:]}_t1ce.nii.gz').get_fdata()
            mid_slice_t1ce = image_t1ce[:,:,90]
            brain.append(np.array(mid_slice_t1ce))
            
        return np.array(seg),np.array(brain)
            

In [2]:
            
load = DataLoader('RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021')


y_train,x_train = load.get_data_train()
x_train = np.array(x_train)
y_train = np.array(y_train)

np.save('x_train',x_train)
np.save('y_train',y_train)

NameError: name 'DataLoader' is not defined

In [3]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')

#przeszukac po publikacjach augmentacje dla MRI (kontrast,przesunięcia)
#sprawdzić imbalans tkanek klas
#przy raportowaniu wynikow raportowac per klasa
x_train.shape

(1200, 240, 240)

In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n,h,w = y_train.shape
y_train_reshape = y_train.reshape(-1,1)
y_train_reshape_encode = labelencoder.fit_transform(y_train_reshape)
y_train_encoded = y_train_reshape_encode.reshape(n,h,w)


y_train = np.expand_dims(y_train_encoded, axis=3)

from keras.utils import normalize
x_train = np.expand_dims(x_train,axis=3)
x_train = normalize(x_train,axis=1)






from sklearn.model_selection import train_test_split
X1,X_test,y1,y_test = train_test_split(x_train,y_train,train_size = 0.8,test_size=0.2) # niezalezny zbior testowy 


X_train,X_do_not,y_train,y_do_not = train_test_split(X1,y1,train_size = 0.8,test_size=0.2) # tutaj podzielic na walidacyjny i testowy testowy ktorego siec nie widzi na zupelnym samym koncu








from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train,num_classes = 4)
y_train_cat = train_masks_cat.reshape((y_train.shape[0],y_train.shape[1],y_train.shape[2],4))



test_masks_cat = to_categorical(y_test,num_classes = 4)
y_test_cat = test_masks_cat.reshape((y_test.shape[0],y_test.shape[1],y_test.shape[2],4))

C:\anaconda3\envs\machine\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
y_train_cat.shape

(768, 240, 240, 4)

In [6]:
X_train.shape

(768, 240, 240, 1)

In [8]:
from keras.models import *
from keras.layers import *
from keras import initializers
from keras.optimizers import *
from keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, Conv2DTranspose, MaxPooling2D, UpSampling2D

import numpy as np 
import os
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

def multi_unet_model(n_classes=4, IMG_HEIGHT=240, IMG_WIDTH=240, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.1)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)

    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    

    return model





model = multi_unet_model(n_classes=4, IMG_HEIGHT=240, IMG_WIDTH=240, IMG_CHANNELS=1)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # jaka funkcja straty (meanIou) 

#sprawdzic learning rate i zwiekszyc dropout
#narzedzie do testowanie https://wandb.ai/site

history = model.fit(X_train, y_train_cat, 
                    batch_size=6, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    #class_weight=class_weights,
                    shuffle=False)
                    

Epoch 1/50
128/128 [==============================] - 17s 57ms/step - loss: 0.1894 - accuracy: 0.9630 - val_loss: 0.1179 - val_accuracy: 0.9726
Epoch 2/50
103/128 [=======================>......] - ETA: 1s - loss: 0.1110 - accuracy: 0.9732

KeyboardInterrupt: 

In [5]:
import keras
model = keras.models.load_model('model_3_channels.h5')


import random
import cv2



test_img_number = random.randint(0, len(X_do_not))
test_img = X_do_not[test_img_number] 
ground_truth=y_do_not[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]

test_img = test_img.reshape(240,240)

from PIL import Image


brain_img = test_img * 255

pred = (predicted_img/predicted_img.max())  * 255

ground_truth = ground_truth.reshape(240,240)

truth =  (ground_truth/ground_truth.max()) * 255



plt.figure(figsize=(30, 20))
plt.subplot(241)
plt.title('Zdjecie mozgu')
plt.imshow(test_img,cmap='gray')
plt.subplot(242)
plt.title('Segmentacja przez eksperta')
plt.imshow(truth,cmap='gray')
plt.subplot(243)
plt.title('Segmentacja przez sieć')
plt.imshow(pred,cmap='gray')

plt.show()

ValueError: in user code:

    File "C:\anaconda3\envs\machine\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\anaconda3\envs\machine\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\anaconda3\envs\machine\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\anaconda3\envs\machine\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\anaconda3\envs\machine\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\anaconda3\envs\machine\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compatibility
        raise ValueError(

    ValueErro